# Packages 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from utils import *


/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Config 

In [4]:
candidate_file_name

'{task}_{data_type}_{model_version}_{model_for_eval}_top{topn}.parquet'

In [5]:
debug = True

eval_cg = False
negative_sampling = 0.1
if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
model_dir = '../model_training'
# train_data_dir = '.'
# test_data_dir = '.'
task = 'task1'
w2v_model = 'w2v_v3'
nic_model = 'nic'
rank_model_version = 'rank_lgbm'

rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100



# # target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")


sub_file = '../data/sub_files/rank_v3.parque'
eval_sub_file = '../data/sub_files/eval_rank_v3.parque'

In [6]:
! mkdir {rank_model_dir}

mkdir: cannot create directory ‘../model_training/rank_lgbm’: File exists


In [7]:
rank_model_dir

'../model_training/rank_lgbm'

In [8]:
# model_dir

# Function 

In [9]:
! ls ../data/candidates/

task1_eval_nic_True_top100.parquet     task1_test_w2v_v3_True_top100.parquet
task1_eval_w2v_v2_top100.parquet       task1_train_nic_True_top100.parquet
task1_eval_w2v_v3_True_top100.parquet  task1_train_w2v_v2_top100.parquet
task1_test_nic_True_top100.parquet     task1_train_w2v_v3_True_top100.parquet
task1_test_w2v_v2_top100.parquet


# Original data 

In [10]:

eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# df_sess.head(3).collect()
test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))


# Eval candidate Generation 

In [11]:
# w2v_file = os.path.join(candidate_path, f'task1_{data_type}_w2v_top100.parquet')
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))

In [12]:
candidate_file_name

'{task}_{data_type}_{model_version}_{model_for_eval}_top{topn}.parquet'

In [13]:
def get_all_candidates(data_type, task=task, 
                       w2v_model=w2v_model, 
                       nic_model=nic_model
                      ,model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn):
    
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    if data_type == 'test':
        original_file_name = f"sessions_{data_type}_{task}.parquet"
    else:
        original_file_name = f"sessions_{data_type}.parquet"
    original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
    w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    
    # get w2v weight
    w2v_pl = w2v_pl.with_columns(
        pl.lit([list(range(w2v_topn, 0, -1))]).alias('w2v_weight')
    )
    get_w2v_weight = pl.element().rank()*0
    nic_pl = nic_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                                # parallel=True
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    cols = ['session_id', 'next_item_prediction', 'w2v_weight']
    combined_pl = w2v_pl.select(cols).join(nic_pl.select(cols), how='left', on='session_id', suffix='_nic')

    combined_pl = combined_pl.with_columns(
        pl.concat_list([pl.col('next_item_prediction'), pl.col('next_item_prediction_nic')]).alias('next_item_prediction'), 
        pl.concat_list([pl.col('w2v_weight'), pl.col('w2v_weight_nic')]).alias('w2v_weight'), 
    ).select(
        pl.all().exclude(['next_item_prediction_nic', 'w2v_weight_nic'])
    )
    # combined_pl = (
    #         pl.concat([w2v_pl.select(cols), nic_pl.select(cols)], how='vertical')
    #             .groupby('session_id')
    #             .agg(
    #                 pl.col('next_item_prediction').flatten()
    #                 , pl.col('w2v_weight').flatten()
    #             )
    # )
    combined_pl = (
        combined_pl.join(original_pl, how='left', on='session_id')
            # .with_columns(
            #     pl.col('prev_items')
            # )
    )
    return combined_pl


In [14]:
train_cg_pl = get_all_candidates(data_type='train')
eval_cg_pl = get_all_candidates(data_type='eval')
test_cg_pl = get_all_candidates(data_type='test')

In [15]:
# train_cg_pl.collect().estimated_size(unit='gb')

In [16]:
# original_pl.collect().estimated_size(unit='gb')

In [17]:




# data_type = 'train'
# w2v_file = os.path.join(candidate_path, 
#                        candidate_file_name.format(
#                 task=task
#             , data_type=data_type
#             , model_version=w2v_model
#             , model_for_eval=model_for_eval
#             , topn=w2v_topn
#                        ))
# nic_file = os.path.join(candidate_path,
#             candidate_file_name.format(
#                 task=task
#                 , data_type=data_type
#                 , model_version=nic_model
#                 , model_for_eval=model_for_eval
#                 , topn=nic_topn
#                        ))
# if data_type == 'test':
#     original_file_name = f"sessions_{data_type}_{task}.parquet"
# else:
#     original_file_name = f"sessions_{data_type}.parquet"
# original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
# w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
# nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))

# # get w2v weight
# w2v_pl = w2v_pl.with_columns(
#     pl.lit([list(range(w2v_topn, 0, -1))]).alias('w2v_weight').cast(pl.List(pl.Int8))
# ).collect()


In [18]:
# get_w2v_weight = pl.element().rank()*0
# nic_pl = nic_pl.with_columns(
#     pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
#                                             # parallel=True
#                                            ).alias('w2v_weight').cast(pl.List(pl.Int8))
# ).collect()

In [19]:
# train_cg_pl.collect()

In [20]:
# eval_cg_pl.collect().estimated_size(unit='gb')

In [21]:
# train_cg_pl.collect()

In [22]:
# print(f"{train_cg_pl.collect().shape}; {eval_cg_pl.collect().shape}; {test_cg_pl.collect().shape}")

In [23]:
# train_cg_pl.collect().head()

In [24]:
# eval_cg_pl.collect()

In [25]:
# get_w2v_weight = pl.element().rank()*0

# train_pl.with_columns(
#     # pl.lit([list(range(100, 0, -1))]).alias('w2v_weight')
#     pl.col('next_item_prediction').arr.lengths().alias('length')
#     , pl.col('next_item_prediction').arr.eval(get_w2v_weight, parallel=True).alias('w2v_weight')
#     # , pl.lit([0 for ele in range(pl.col('length'))])

# ).head().collect()

In [26]:
# dir(pl.element())

In [27]:
# w2v_pl.head().collect()


In [28]:

# def get_all_candidates(data_type):
#     w2v_file = os.path.join(candidate_path, f'task1_{data_type}_w2v_top100.parquet')
#     nic_file = os.path.join(candidate_path, f'task1_{data_type}_nic_top100.parquet')
#     w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
#     nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))

#     cols_to_keep = ['session_id', 'next_item_prediction', 'cg_source']

#     w2v_pl = w2v_pl.with_columns(pl.lit('word2vec').alias('cg_source')).select(cols_to_keep).explode('next_item_prediction')
#     nic_pl = nic_pl.with_columns(pl.lit('next_item_counter').alias('cg_source')).select(cols_to_keep).explode('next_item_prediction')
#     # length = w2v_pl.collect().shape[0]
#     # w2v_pl = w2v_pl.with_columns(
#     #     pl.Series(values=[list(range(100, 0, -1))]*length, name='w2v_weight')
#     # )
#     cg_pl = (
#             pl.concat([w2v_pl, nic_pl], how='vertical')
#                 .groupby(['session_id', 'next_item_prediction'])
#                 .agg(
#                     pl.col('cg_source')
#                 )
#                 .with_columns(
#                     pl.when(pl.col('cg_source').arr.contains(pl.lit('word2vec'))).then(1).otherwise(0).alias('whether_w2v')
#                     , pl.when(pl.col('cg_source').arr.contains(pl.lit('next_item_counter'))).then(1).otherwise(0).alias('whether_nic')
#                 )
#                 .select(
#                     pl.all().exclude('cg_source')
#                 )
#     )
#     return cg_pl

## Eval candidate generation 

In [29]:
if eval_cg:
    eval_topn = 300

    col = f"recall@{eval_topn}"

    eval_final = (
            eval_cg_pl
            .lazy()
            .with_columns(
                pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
            )
            .with_columns(
                pl.concat_list([pl.col('next_item'), pl.col('next_item_prediction')]).alias('mrr')
            )
            ).with_columns(
                pl.col('next_item_prediction').arr.head(eval_topn).arr.contains(pl.col('next_item')).mean().alias(col)

            )
    final_res = eval_final.select(
            pl.count().alias('total_sessions')
            , pl.col(col).mean()

        ).collect()
    print(final_res)
    del final_res

In [30]:
# eval_cg_pl.head().collect()

# Feature Process 

## Load Model 

In [31]:
nic_model = (
    pl.scan_parquet('../model_training/next_item_counter_v2/nic_model.parquet')
        .explode(['next_item_prediction', 'next_item_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
        )
            )
print(nic_model.schema)



# w2v_model_file = '../model_training/v2/w2v.model'
w2vec = Word2Vec.load(w2v_model_file)
annoy_index = AnnoyIndexer()
annoy_index.load(annoy_index_file)

{'next_item_prediction': Utf8, 'next_item_weight': Float64, 'last_prev_item': Utf8}


In [32]:
# nic_model

In [33]:
# len(w2vec.wv)

In [34]:
# train_cg_pl.schema

In [35]:
# w2vec.wv.similarity('2', '3')

In [36]:
# train_cg_pl = train_cg_pl.collect()

In [37]:
# train_cg_pl.shape

In [38]:
# train_cg_pl.head()

In [39]:
# target_df.head().collect()

In [40]:
# pos.head()/collect()

In [41]:
# def get_w2v_simi(x):
#     try:
#         simi = w2vec.wv.similarity(x['next_item_prediction'],
#                                                   x['last_prev_item']
#                                                  )
#     except:
#         simi = 0
#     return simi

In [42]:
# target_df.head().collect()

In [43]:
# target_df = target_df.with_columns(
#                 pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
#                 , pl.struct(["next_item_prediction", "last_prev_item"]).apply(
#                     lambda x: get_w2v_simi(x)).alias('last_item_similarity')
#                 , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
#                     .then(2)
#                     .otherwise(3).alias('locale')
#             ).sort('session_id')

In [89]:
# dir(w2vec.wv)

In [94]:
w2vec.wv.n_similarity(["B09W9FND7K", "B09JSPLN1M"], ["B09M7GY217"])

0.7246886

In [114]:
# target_df = train_candidates
# data_type = 'train'

# def get_w2v_simi(x, col='last_prev_item'):
#     try:
#         simi = w2vec.wv.similarity(x['next_item_prediction'],
#                                                   x[col]
#                                                  )
#     except Exception as e:
#         print(e)
#         # print(x[col])
#         simi = 0
#     return simi

def get_w2v_simi(x, col='last_prev_item'):
    try:
        if isinstance(x[col], str):
            target = [x[col]]
        else:
            target = x[col]
        simi = w2vec.wv.n_similarity([x['next_item_prediction']],
                                                  target
                                                 )
    except Exception as e:
        print(e)
        # print(x[col])
        simi = 0
    return simi

def get_feature(target_df, data_type, nic_model=nic_model):
    # if data_type == 'train':
    #     basic_info_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_train.parquet'))
    # else:
    #     basic_info_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'))
    # basic_info_pl = basic_info_pl.with_columns(
    #         pl.col('prev_items').apply(str2list)
    #     )
    # print(f"basci_info_pl:")
    # print(basic_info_pl.schema)
    # nic model 

    # target_df = (
    #     target_df.explode(['next_item_prediction', 'w2v_weight'])
    #     .with_column(
    #             pl.col('prev_items').arr.get(-1).alias('last_prev_item')
    #         )
    #         .join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
    #         .with_columns(
    #             pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
    #             , pl.struct(["next_item_prediction", "last_prev_item"]).apply(
    #                 lambda x: get_w2v_simi(x)).alias('last_item_similarity')
    #             , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
    #                 .then(2)
    #                 .otherwise(3).alias('locale')
    #         ).sort('session_id')
    # )
    # if data_type != 'test':
    #     target_df = (
    #         target_df
    #             .with_columns(
    #                 pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
    #             )
    #     )
# target_df = train_cg_pl


    # data_type = 'train'
    target_df = target_df.explode(['next_item_prediction', 'w2v_weight'])

    if data_type != 'test':
        target_df = (
            target_df
                .with_columns(
                    pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
                )
        )

    if data_type == 'train':
        target_df = target_df.collect()
        pos = target_df.filter(pl.col('target')==1)
        neg = target_df.filter(pl.col('target')==0).sample(frac=negative_sampling)
        target_df = pl.concat([pos, neg], how='vertical')
        print(target_df.select('target').mean())

    target_df = (
            target_df.lazy().with_columns(
                    pl.col('prev_items').arr.get(-1).alias('last_prev_item')
                ).join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
                .with_columns(
                    pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
                    , pl.struct(["next_item_prediction", "last_prev_item"]).apply(
                        lambda x: get_w2v_simi(x, col='last_prev_item')).alias('last_item_similarity').cast(pl.Float32)
                    , pl.struct(["next_item_prediction", "prev_items"]).apply(
                        lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
                    , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                        .then(2)
                        .otherwise(3).alias('locale')
                ).sort('session_id')

    )
    return target_df
# target_df.head(3).collect()

In [115]:
train_candidates = get_feature(target_df=train_cg_pl, data_type='train')

shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.097466 │
└──────────┘


In [119]:
train_candidates.collect().sample(20)

session_id,next_item_prediction,w2v_weight,prev_items,next_item,locale,target,last_prev_item,next_item_weight,last_item_similarity,prev_item_similarity
i64,str,i64,list[str],str,i32,i32,str,f64,f32,f32
938,"""B086QBX758""",85,"[""B00IU74F6U"", ""B09KD5JRDQ""]","""B091TWDXXB""",1,0,"""B09KD5JRDQ""",0.0,0.877628,0.942518
335,"""B003ODMCJ2""",59,"[""B00DIW0FP2"", ""B00E67G5X4"", ""B084G4MJPB""]","""B01DO5185S""",1,0,"""B084G4MJPB""",0.0,0.928818,0.943675
219,"""B08XY6PJB6""",42,"[""B084KSHVBQ"", ""B084KSHVBQ""]","""B00LZI47N2""",1,0,"""B084KSHVBQ""",0.0,0.982158,0.982158
161,"""B07QR85J7B""",0,"[""B08X3HCW5Q"", ""B09H6DC9Z5"", … ""B09G386KLZ""]","""B07QR85J7B""",1,1,"""B09G386KLZ""",0.01,0.92772,0.965777
386,"""B09RMYYJ3R""",26,"[""B098DXKB31"", ""B08DPLF6K9""]","""B08DPNG346""",1,0,"""B08DPLF6K9""",0.0,0.761656,0.857338
129,"""B09Q8FPGKX""",86,"[""B08R9T375R"", ""B09F5L7VX9"", ""B07Y1YMXFY""]","""B07Y1TZMWZ""",1,0,"""B07Y1YMXFY""",0.0,0.83288,0.929354
131,"""B097TN4X9F""",72,"[""B003BU3XRS"", ""B003BU3XRS""]","""B07Y69W9GX""",1,0,"""B003BU3XRS""",0.0,0.956873,0.956873
548,"""B07G5NNPJV""",0,"[""B08P91WG55"", ""B07LBMVKW8"", ""B07H5K9SYS""]","""B0071LQ1DQ""",1,0,"""B07H5K9SYS""",0.01,0.822309,0.835763
916,"""B007N3BDC6""",0,"[""B07WS1GQLK"", ""B09RMNCMHW"", ""B0753929DV""]","""B07538S1M9""",1,0,"""B0753929DV""",0.01,0.835747,0.849841


In [53]:
eval_candidates = get_feature(target_df=eval_cg_pl, data_type='eval')

/var/tmp/ipykernel_13514/4243057303.py:69: DeprecationWarning: `LazyFrame.with_column` has been renamed; this redirect is temporary, please use `.with_columns` instead
  target_df.lazy().with_column(


In [46]:
# eval_candidates.collect()

In [47]:
# target_df = eval_cg_pl
# data_type = 'eval'
# (
#     target_df.explode(['next_item_prediction', 'w2v_weight'])
#     .with_column(
#             pl.col('prev_items').arr.get(-1).alias('last_prev_item')
#         )
#         .join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
#         .with_columns(
#             pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
#             # , pl.struct(["next_item_prediction", "last_prev_item"]).apply(
#             #     lambda x: w2vec.wv.similarity(x['next_item_prediction'],
#             #                                   x['last_prev_item']
#             #                                  )).alias('last_item_similarity')
#             , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
#                 .then(2)
#                 .otherwise(3).alias('locale')
#         ).sort('session_id')
# ).head().collect()
# # if data_type != 'test':
# #     target_df = (
# #         target_df
# #             .with_columns(
# #                 pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
# #             )
# #     )
# # target_df.head().collect()

In [48]:
# eval_candidates.head().collect()

In [49]:
# test_candidates.collect()

In [50]:
# test_candidates.head().collect()

# Ranker 

## Config 

In [54]:
estimator = 10
if debug:
    estimator = 3
    
target = 'target'
feature_cols = [
    # 'whether_w2v'
    # , 'whether_nic'
    'next_item_weight'
    , 'locale'
    , 'w2v_weight'
    , 'last_item_similarity'
]

categorical_feature=['locale']

In [52]:
# train_candidates.head().collect()[feature_cols + [target]]

## Train model

In [53]:
# train_candidates = train_candidates.sort('session_id')


# .head(100).collect()

In [54]:
ranker = LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=estimator, 
    importance_type='gain',
    eval_at=[5]
)

In [55]:
train_candidates = train_candidates.collect()
train_candidates.shape

(39883137, 10)

In [56]:
print(train_candidates.select('target').mean())


shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.100714 │
└──────────┘


In [57]:
eval_candidates = eval_candidates.collect()
eval_candidates.shape

(40019694, 10)

In [58]:
print(eval_candidates.select('target').mean())


shape: (1, 1)
┌──────────┐
│ target   │
│ ---      │
│ f64      │
╞══════════╡
│ 0.006248 │
└──────────┘


In [59]:
session_lengths_train = train_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()
session_lengths_eval = eval_candidates.groupby('session_id').agg(pl.count()).select('count').to_pandas()['count'].values.tolist()

In [60]:
ranker = ranker.fit(
    X=train_candidates[feature_cols].to_pandas(),
    y=train_candidates[target].to_pandas(),
    group=session_lengths_train,
    eval_set=[(train_candidates[feature_cols].to_pandas(), train_candidates[target].to_pandas()),
           (eval_candidates[feature_cols].to_pandas(), eval_candidates[target].to_pandas())
             ],
    eval_group=[session_lengths_train,
                session_lengths_eval
               ]
    , categorical_feature=categorical_feature
    # , early_stopping_rounds=
)

/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/sklearn.py:621: UserWarning: Found 'eval_at' in params. Will use it instead of 'eval_at' argument
  _log_warning(f"Found '{alias}' in params. Will use it instead of 'eval_at' argument")
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:2068: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['locale']
  _log_warning('categorical_feature in Dataset is overridden.\n'
/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overridin

[1]	valid_0's ndcg@5: 0.859077	valid_1's ndcg@5: 0.770728
[2]	valid_0's ndcg@5: 0.85267	valid_1's ndcg@5: 0.769357
[3]	valid_0's ndcg@5: 0.845334	valid_1's ndcg@5: 0.771173
[4]	valid_0's ndcg@5: 0.84483	valid_1's ndcg@5: 0.771243
[5]	valid_0's ndcg@5: 0.841044	valid_1's ndcg@5: 0.771131
[6]	valid_0's ndcg@5: 0.840387	valid_1's ndcg@5: 0.771163
[7]	valid_0's ndcg@5: 0.839692	valid_1's ndcg@5: 0.771436
[8]	valid_0's ndcg@5: 0.839389	valid_1's ndcg@5: 0.771334
[9]	valid_0's ndcg@5: 0.839531	valid_1's ndcg@5: 0.771612
[10]	valid_0's ndcg@5: 0.838751	valid_1's ndcg@5: 0.771461


In [61]:
print(train_candidates.estimated_size(unit='gb'))
del train_candidates

6.670078314840794


In [62]:
print(eval_candidates.estimated_size(unit='gb'))
del eval_candidates

6.703331420198083


## Save model 

In [64]:
joblib.dump(
        value=ranker,
        filename=os.path.join(rank_model_dir, 'model.pkl')
)

['../model_training/rank_lgbm/model.pkl']

In [ ]:
del ranker

## Load Model 

In [55]:
ranker = joblib.load(os.path.join(rank_model_dir, 'model.pkl'))
ranker

LGBMRanker(boosting_type='dart', eval_at=[5], importance_type='gain',
           metric='ndcg', n_estimators=10, objective='lambdarank')

### Importance 

In [3]:
# ranker

LGBMRanker(eval_at=[3], importance_type='gain', metric='ndcg',
           n_estimators=1000, objective='lambdarank')

In [4]:
# ranker = joblib.load(os.path.join('../model_training/rank_lgbm_v2/', 'model.pkl'))
impotant_df = pd.DataFrame(
    {
        'features': ranker.feature_name_,
        'importance': ranker.feature_importances_
    }
).sort_values('importance', ascending=False)
impotant_df

,features,importance
0,next_item_weight,1.259315e+07
3,last_item_similarity,1.358875e+05
4,prev_item_similarity,1.982588e+04
2,w2v_weight,1.630157e+04
5,prev_length,9.391630e+02
1,locale,0.000000e+00


# Eval ranker 

In [57]:
def rank_rec(candidate_pl, feature_cols, ranker, topn=100):

    candidate_pl = candidate_pl.collect()
    print(f"candidate pl shape: {candidate_pl.shape}")
    inference = ranker.predict(candidate_pl[feature_cols].to_pandas())
    test_result = (candidate_pl
         .lazy()
         .with_columns(
             pl.Series(name='predict', values=inference)
         )
         .with_columns(
            pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
         )
         .sort(['session_id', 'rank'])
         .filter(pl.col('rank')<=topn)
         .groupby(['session_id'])
         .agg(
             pl.col('next_item_prediction')
             , pl.col('next_item').unique()#.arr.get(0).cast(pl.Utf8)
         ).with_columns(
             pl.col('next_item').arr.get(0).alias('next_item')
         )
    )
    return test_result


In [58]:
ranker_eval_pl = rank_rec(candidate_pl=eval_candidates,
                         feature_cols=feature_cols
                         , ranker=ranker
                        # , topn=10
                         )

candidate pl shape: (40019694, 10)


In [59]:
# ranker_eval_pl.collect().shape

(326443, 3)

In [60]:
model_eval(target_df=eval_pl.join(ranker_eval_pl.lazy(), how='left', on='session_id'))

total_sessions,mrr,recall@20,recall@100
u32,f64,f64,f64
326443,0.364766,0.475869,0.52456


In [64]:
ranker_eval_pl.head().collect()

session_id,next_item_prediction,next_item
i64,list[str],str
3,"[""B0B71GSJ2R"", ""B0B71KXCSB"", … ""B09TW3GVT3""]","""B0B4R9NN4B"""
4,"[""B09FKDPY2S"", ""B09MRVWQ3B"", … ""B099S1WHRQ""]","""B0BGVBKWGZ"""
14,"[""B07LFQPX5L"", ""B07LFR6J9M"", … ""B08WHFQZWF""]","""B07LFRP5SS"""
15,"[""B0773V43RV"", ""B000VVSWBM"", … ""B07LG41SN6""]","""B000VVSWBM"""
27,"[""B08XBNRD25"", ""B09ZPNZXQ5"", … ""B07DQCXS4P""]","""B09ZPNHQ12"""


In [1]:
# ranker_eval_pl.with_columns(
#     pl.col('next_item_prediction').arr.lengths().alias('length')
#     , pl.col('next_item_prediction').arr.unique().arr.lengths().alias('unique_length')
# ).select(pl.col('length').sub(pl.col('unique_length')).alias('diff')).collect().to_series().value_counts()

In [67]:
ranker_eval_pl.collect().write_parquet(eval_sub_file)

# Test inference 

In [77]:
test_cg_pl.schema

{'session_id': Int64,
 'next_item_prediction': List(Utf8),
 'w2v_weight': List(Int64),
 'prev_items': Unknown,
 'locale': Utf8}

In [87]:
# nic_model

In [88]:
test_candidates = get_feature(target_df=test_cg_pl, data_type='test')

In [89]:
test_candidates = test_candidates.collect()
test_candidates.shape

(41665715, 8)

In [90]:
inference = ranker.predict(test_candidates[feature_cols].to_pandas())

In [91]:
inference

array([-1.25055664, -0.00440581, -1.25055664, ...,  0.8638483 ,
        0.12114125,  0.1000968 ])

In [2]:
# test_candidates.head()

In [93]:
test_result = (test_candidates
     .lazy()
     .with_columns(
         pl.Series(name='predict', values=inference)
     )
     .with_columns(
        pl.col('predict').rank(method='ordinal', descending=True).over('session_id').alias('rank')
     )
     .sort(['session_id', 'rank'])
     .filter(pl.col('rank')<=100)
     .groupby(['session_id'])
     .agg(
         pl.col('next_item_prediction')
     )
)
# test_result.head().collect()

In [94]:
test_pl.schema

{'prev_items': Unknown, 'locale': Utf8, 'session_id': Int64}

In [73]:
predictions = test_pl.join(test_result, how='left', on='session_id').collect()[['locale', 'next_item_prediction']].to_pandas()

In [74]:
predictions.shape

(316971, 2)

In [75]:
predictions.shape

(316971, 2)

In [98]:
test_result.tail().collect()

session_id,next_item_prediction
i64,list[str]
3923215,"[""B08X9L5RGD"", ""B07V5FL8G6"", … ""B0B4WXMX82""]"
3923216,"[""B0989BHLSY"", ""B09CPNS7XV"", … ""B08PC1FRT8""]"
3923217,"[""B09HKZBNZH"", ""B09HZSRJWW"", … ""B002SLEKCI""]"
3923218,"[""B07TR5LQSL"", ""B07TR5LQSL"", … ""B09TFB3XXG""]"
3923219,"[""B09XPX59JK"", ""B09XPX59JK"", … ""B084PB8DKV""]"


In [99]:
test_pl.tail().collect()

prev_items,locale,session_id
list[str],str,i64
"[""B077SZ2C3Y"", ""B0B14M3VZX""]","""UK""",3923215
"[""B08KFHDPY9"", ""B0851KTSRZ"", … ""B081YDH55K""]","""UK""",3923216
"[""B07PY1N81F"", ""B07Q1Z8SQN"", … ""B09HL11V5B""]","""UK""",3923217
"[""B01MCQMORK"", ""B09JYZ325W""]","""UK""",3923218
"[""B0B8JX92YJ"", ""B09TN4MP6V"", ""B0BG2LZQSL""]","""UK""",3923219


In [77]:
# original_test.head()

In [78]:
# ! ls -al ../data/sub_files/

In [79]:
check_predictions(predictions, test_sessions=test_pl.collect().to_pandas(), 
                  # check_products=True, product_df=products
                 )
# Its important that the parquet file you submit is saved with pyarrow backend
predictions.to_parquet(sub_file, engine='pyarrow')

In [81]:
# You can submit with aicrowd-cli, or upload manually on the challenge page.
!aicrowd submission create -c task-1-next-product-recommendation -f {sub_file}

rank_v3.parque ━━━━━━━━━━━━━━━━━━━━ 100.0% • 285.8/285.7 MB • 2.9 MB/s • 0:00:0000:0100:05
                                                                                 ╭─────────────────────────╮                                                                                 
                                                                                 │ Successfully submitted! │                                                                                 
                                                                                 ╰─────────────────────────╯                                                                                 
                                                                                       Important links                                                                                       
┌──────────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

# submission file check 

In [7]:
sub_df = pl.scan_parquet(sub_file)
test_df = pl.scan_csv('../data/raw_data/sessions_test_task1.csv')

In [8]:
sub_df.schema

{'locale': Utf8, 'next_item_prediction': List(Utf8)}

In [9]:
test_df.schema

{'prev_items': Utf8, 'locale': Utf8}

In [12]:
sub_df.tail().collect()

locale,next_item_prediction
str,list[str]
"""UK""","[""B08X9L5RGD"", ""B07V5FL8G6"", … ""B0B4WXMX82""]"
"""UK""","[""B0989BHLSY"", ""B09CPNS7XV"", … ""B08PC1FRT8""]"
"""UK""","[""B09HKZBNZH"", ""B09HZSRJWW"", … ""B002SLEKCI""]"
"""UK""","[""B07TR5LQSL"", ""B07TR5LQSL"", … ""B09TFB3XXG""]"
"""UK""","[""B09XPX59JK"", ""B09XPX59JK"", … ""B084PB8DKV""]"


In [22]:
sub_df.with_columns(
    pl.col('next_item_prediction').arr.lengths().alias('length')
    , pl.col('next_item_prediction').arr.unique().arr.lengths().alias('unique_length')
).select('unique_length').collect().to_series().

unique_length,counts
u32,u32
96,20664
64,74
63,58
95,18978
62,43
94,17508
93,16207
61,21
92,14685


In [11]:
test_df.tail().collect()

prev_items,locale
str,str
"""['B077SZ2C3Y' …","""UK"""
"""['B08KFHDPY9' …","""UK"""
"""['B07PY1N81F' …","""UK"""
"""['B01MCQMORK' …","""UK"""
"""['B0B8JX92YJ' …","""UK"""
